## Add available processors

In [ ]:
using Distributed
nprocs()

In [ ]:
nprocs() == 1 && addprocs(3)

In [ ]:
workers()

## Remote Calls and References

In [ ]:
ref = @spawn sin(π/5)

In [ ]:
fetch(ref)

In [ ]:
n=10
refs = Array{Future,1}(undef,n)
for i = 1:n
    refs[i] = @spawn sin(i)
end

In [ ]:
refs

In [ ]:
reduce(+,map(fetch,refs))

In [ ]:
res=@distributed (vcat) for i=1:n
    sin(i)
end

In [ ]:
reduce(+,res)

In [ ]:
@everywhere using DataFrames
res=@distributed (vcat) for i = 1:10
    println((i,sin(i)))
    DataFrame(worker=myid(), vals=sin(i))
end
res

In [ ]:
sum(res.vals)

## Monte-Carlo Simulation to estimate $\pi$

In [ ]:
#==========================#
# monte-carlo simulation
# π r^2 / 4 r^2 = s/n 
#==========================#


@everywhere function isInside() 
    x = rand()
    y = rand()
    x^2 + y^2 < 1 ? 1 : 0
end;

@everywhere function ppi(n)
    s=@distributed (+) for i = 1:n
        isInside()
    end
    4s/n
end;

function pi(n)
    s=0.0
    for i = 1:n
        s+=isInside()
    end
    4s/n
end;


In [ ]:
@time ppi(10^9)

In [ ]:
@time pi(10)

## Distributed Arrays

In [ ]:
@everywhere using DistributedArrays

In [ ]:
a = round.(10*rand(20,20))

In [ ]:
map(x->myid(),a)

In [ ]:
da = distribute(a)

In [ ]:
map(x->myid(),da)

In [ ]:
da.pids

In [ ]:
[@spawnat p sum(localpart(da)) for p in da.pids]

In [ ]:
map(fetch,[@spawnat p sum(localpart(da)) for p in da.pids])

In [ ]:
reduce(+,map(fetch,[@spawnat p sum(localpart(da)) for p in da.pids]))

In [ ]:
sum(da)

In [ ]:
@code_typed sum(da)

## Cross-validation in parallel

In [ ]:
@everywhere using RDatasets
@everywhere using Statistics
@everywhere using DecisionTree

@everywhere function irisAcc() 
    iris = dataset("datasets", "iris")
    features = iris[:, 1:4] |>Matrix;
    labels = iris[:, 5] |> Vector{String};
    model = build_forest(labels, features, 2, 4, 0.5, 6);
    accuracy = nfoldCV_forest(labels, features, 2, 4, 2, 0.5);
    mean(accuracy)
end

In [ ]:
function mserial(n)
    sm=0.0
    for i=1:n
         sm += irisAcc()
    end
    print("Overall Acc:",sm/n*100.0)
end
@time mserial(100)

In [ ]:
function mparallel(n)
    s=@distributed (+) for i=1:n
        irisAcc()
    end
    print("Overall Acc:",s/n*100.0)
end
@time mparallel(100)